<a href="https://colab.research.google.com/github/Aashish1106/Text-Generation/blob/test/Text_Generation_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Text Generation** using a **Recurrent Neural Networks**, specifically a **Long Short-Term Memory Network**, implementing this network in Python, and use it to generate some text.

In [3]:
import numpy
import sys
import nltk 
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
file = open("New.txt").read()

In [5]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [6]:
# preprocess the input data, make tokens
processed_inputs = tokenize_words(file)

In [7]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [8]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 2376
Total vocab: 53


In [9]:
seq_length = 100
x_data = []
y_data = []

In [10]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [11]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 2276


In [12]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [13]:
y = np_utils.to_categorical(y_data)

In [14]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [17]:
model.fit(X, y, epochs=205, batch_size=256, callbacks=desired_callbacks)

Epoch 1/205
9/9 [==============================] - 31s 3s/step - loss: 3.4769

Epoch 00001: loss improved from inf to 3.47691, saving model to model_weights_saved.hdf5
Epoch 2/205
9/9 [==============================] - 26s 3s/step - loss: 3.0860

Epoch 00002: loss improved from 3.47691 to 3.08604, saving model to model_weights_saved.hdf5
Epoch 3/205
9/9 [==============================] - 27s 3s/step - loss: 3.0418

Epoch 00003: loss improved from 3.08604 to 3.04175, saving model to model_weights_saved.hdf5
Epoch 4/205
9/9 [==============================] - 27s 3s/step - loss: 3.0274

Epoch 00004: loss improved from 3.04175 to 3.02735, saving model to model_weights_saved.hdf5
Epoch 5/205
9/9 [==============================] - 27s 3s/step - loss: 2.9988

Epoch 00005: loss improved from 3.02735 to 2.99880, saving model to model_weights_saved.hdf5
Epoch 6/205
9/9 [==============================] - 27s 3s/step - loss: 2.9932

Epoch 00006: loss improved from 2.99880 to 2.99318, saving model 

In [18]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [20]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" ng information readable state unintelligible nonsense sender encrypted message shares decoding techn "


In [22]:
# generate the text
for i in range(1000):
  x = numpy.reshape(pattern, (1,len(pattern), 1))
  x = x/float(vocab_len)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[ 1:len(pattern)]

rtication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovertication aomputing pooor iovert

KeyboardInterrupt: ignored